# LSTM Stock Predictor

In [2]:
import numpy as np
import pandas as pd
import hvplot.pandas
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [ ]:
# Load the fear and greed sentiment data for Bitcoin

#start_date = '2017-01-01'
#end_date = '2019-12-31'

df = pd.read_csv('btc_historic.csv', index_col='Date', infer_datetime_format=True, parse_dates=True)
df = df.drop(columns=['High','Low','Close', 'Volume'])
df.head()

In [3]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [4]:
# Predict Closing Prices using a 5 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 5

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 0
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

# Use 80% of the data for training and the remaineder for testing
split = int(.8 * len(X))
X_train = X[:split - 1]
X_test = X[split:]
y_train = y[:split - 1]
y_test = y[split:]

In [5]:
# Use the MinMaxScaler to scale data between 0 and 1.
x_train_scaler = MinMaxScaler()
x_test_scaler = MinMaxScaler()
y_train_scaler = MinMaxScaler()
y_test_scaler = MinMaxScaler()

# Fit the scaler for the training Data
x_train_scaler.fit(X_train)
y_train_scaler.fit(y_train)

# Scale the training data
X_train = x_train_scaler.transform(X_train)
y_train = y_train_scaler.transform(y_train)

# Fit the scaler for the testing Data
x_test_scaler.fit(X_test)
y_test_scaler.fit(y_test)

# Scale the y_test data
X_test = x_test_scaler.transform(X_test)
y_test = y_test_scaler.transform(y_test)

# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [6]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features
model = Sequential()
model.add(LSTM(
    units=30, return_sequences=True,
    input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.5))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=30))
model.add(Dropout(0.5))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 30)             3840      
_________________________________________________________________
dropout (Dropout)            (None, 5, 30)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 30)             7320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 30)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3

In [7]:
# Train the model
# Use at least 100 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
model.fit(X_train, y_train, epochs=200, shuffle=False, batch_size=1, verbose=1)

Epoch 1/200
597/597 [==============================] - 12s 5ms/step - loss: 0.0084
Epoch 2/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0533
Epoch 3/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0587
Epoch 4/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0399
Epoch 5/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0250
Epoch 6/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0153
Epoch 7/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0099
Epoch 8/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0129
Epoch 9/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0101
Epoch 10/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0108
Epoch 11/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0103
Epoch 12/200
597/597 [==============================] - 3s 5ms/step - loss: 0.0101
Epoch 13/200

---

In [8]:
# Evaluate the model
model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 4ms/step - loss: 0.0241


0.02409171499311924

In [9]:
# Make some predictions
predicted = model.predict(X_test)

# Recover the original prices instead of the scaled version
predicted_prices = y_test_scaler.inverse_transform(predicted)
real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

# Create a DataFrame of Real and Predicted values
btc = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()}, 
    index = df.index[-len(real_prices): ]) 
btc.head()

,Real,Predicted
Date,,
2019-05-30,269.272980,281.522949
2019-05-31,270.008392,279.885590
2019-06-03,266.369965,278.462646
2019-06-04,265.692596,276.616882
2019-06-05,271.459900,274.379578


In [10]:
# Plot the real vs predicted values as a line chart
btc.head(100).hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)